In [ ]:
import os
import sys
sys.path.append(os.getcwd() + '/..')
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from simcse import SimCSE
from tqdm.notebook import tqdm
from sklearn.manifold import TSNE
np.random.seed(114514)
torch.manual_seed(114514)

In [ ]:
model_path = './../models/retrieval/entity_type_tuned_sota/'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SimCSE(model_path,device=device)

In [ ]:
cat_data = pd.read_csv('./../data/raw/Cat_data.csv')
mappings_data = pd.read_csv('./../data/raw/Aggregated_siblings_data.csv')
neg = mappings_data[mappings_data['Parent Label'] == 'All products']
mappings_data = mappings_data.drop(neg.index)
large_index = mappings_data[mappings_data['Cluster size'] > 60].index
small_index = mappings_data[mappings_data['Cluster size'] == 4].index[:60]
classlabels = list(cat_data['Label'])
embeddings = model.encode(classlabels)
model.build_index(classlabels)
knn = model.search(classlabels,threshold=0,top_k=int(mappings_data['Cluster size'].max()))

In [ ]:
catid_dict = {}
idx_dict = {}
for i,row in cat_data.iterrows():
    catid_dict[row['Cat ID']]=i
    idx_dict[i]=row['Cat ID']
def idx_to_catid(x):
    return idx_dict[x]
def catid_to_idx(cid):
    return catid_dict[cid]

In [ ]:
def evaluate(model,class_data,mappings_data,precomputed_knn=None,weighted=False,weight='Linear',repeat:int=10):
    if precomputed_knn:
        knn = precomputed_knn
    else:
        print("Computing k-NN:")
        k = 1+4*mappings_data['Cluster size'].max()
        classlabels = list(class_data['Label'])
        model.build_index(classlabels)
        knn = model.search(classlabels,threshold=0,top_k=int(k))
    print("Evaluating metrics:")
    cluster_ids = mappings_data['Cluster ID'].unique()
    M = len(cluster_ids)
    pltdomain = np.arange(0.1,2.1,0.1)
    hits = np.zeros([pltdomain.shape[0],M,repeat])
    with tqdm(total = repeat, position = 0, desc = 'Repetition') as outer:
        with tqdm(total = M, position = 1, leave = False, desc = 'Cluster') as inner:
            for r in range(repeat):
                inner.reset()
                for j,c in enumerate(cluster_ids):
                    cluster = mappings_data[mappings_data['Cluster ID'] == c]
                    randclass = cluster.sample(n=1)['Cat ID'].iloc[0]
                    ref_cluster_ids = mappings_data[mappings_data['Cat ID'] == randclass]['Cluster ID'].unique()
                    reference = set(mappings_data[mappings_data['Cluster ID'].isin(ref_cluster_ids)]['Cat ID'])
                    n = len(reference)
                    predictions = [idx_to_catid(entry[0]) for entry in knn[catid_to_idx(randclass)]]
                    for i,x in enumerate(pltdomain):
                        sublist = predictions[:int(np.ceil(x*n))]
                        hits[i][j][r] = len(set(sublist).intersection(reference)) / n
                    inner.update(1)
                outer.update(1)
    recall = np.mean(hits,axis=(1,2))
    err = 1.96*np.std(np.mean(hits,axis=1),axis=1)/np.sqrt(repeat)
    print(f"Hit @ N/2:{recall[4]:.4f}\nHit @ N:{recall[9]:.4f}\nHit @ 2N:{recall[19]:.4f}\n")
    fig, ax = plt.subplots()
    ax.errorbar(pltdomain, recall, yerr=err)
    plt.xlabel("Sampling ratio")
    plt.ylabel("Recall")
    plt.show()
    return hits

In [ ]:
eval_details = evaluate(model,cat_data,mappings_data,precomputed_knn=knn)

In [ ]:
randseed = np.random.choice(len(classlabels),1).item()
num_candidates = 15
knn_results = [(idx_to_catid(entry[0]),entry[1],entry[2]) for entry in knn[randseed][:num_candidates+1]]
ref_clusters = mappings_data[mappings_data['Cat ID'] == idx_to_catid(randseed)]['Cluster ID'].unique()
reference = mappings_data[mappings_data['Cluster ID'].isin(ref_clusters)]
ref_id = set(reference['Cat ID'])
predictions = pd.DataFrame([(name, sim, 'Seed' if catid == idx_to_catid(randseed) else catid in ref_id) for catid, name, sim in knn_results],columns=['Category','Similarity','In reference'])
display(predictions)
display(reference)

In [ ]:
# embed_tsne3 = TSNE(n_components=3, learning_rate='auto',init='random', perplexity=50).fit_transform(embeddings.numpy())
embed_tsne2 = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=50).fit_transform(embeddings.numpy())

In [ ]:
large_clusters = embed_tsne2[large_index]
x = large_clusters[:,0]
y = large_clusters[:,1]
color = np.array(mappings_data['Cluster ID'][large_index])
fig,ax = plt.subplots(figsize=(8,8))
ax.scatter(x,y,s=6,c=color)
ax.set_title('Large clusters')
plt.show()

In [ ]:
small_clusters = embed_tsne2[small_index]
x = small_clusters[:,0]
y = small_clusters[:,1]
color = np.array(mappings_data['Cluster ID'][small_index])
fig,ax = plt.subplots(figsize=(8,8))
ax.scatter(x,y,s=6,c=color)
ax.set_title('Small clusters')
plt.show()

In [ ]:
# large_clusters3 = embed_tsne3[large_index]
# x = large_clusters3[:,0]
# y = large_clusters3[:,1]
# z = large_clusters3[:,2]
# color = np.array(mappings_data['Cluster ID'][large_index])
# fig = plt.figure(figsize = (8,8))
# ax = fig.add_subplot(projection='3d')
# ax.scatter(x,y,z,s=6,c=color)
# ax.set_title('Large clusters')
# plt.show()

In [ ]:
# small_clusters3 = embed_tsne3[small_index]
# x = small_clusters3[:,0]
# y = small_clusters3[:,1]
# z = small_clusters3[:,2]
# color = np.array(mappings_data['Cluster ID'][small_index])
# fig = plt.figure(figsize = (8,8))
# ax = fig.add_subplot(projection='3d')
# ax.scatter(x,y,z,s=6,c=color)
# ax.set_title('Small clusters')
# plt.show()

In [ ]:
num_candidates = 35
knn_results = [(idx_to_catid(entry[0]),entry[1],entry[2]) for entry in knn[randseed][:num_candidates+1]]
ref_clusters = mappings_data[mappings_data['Cat ID'] == idx_to_catid(randseed)]['Cluster ID'].unique()
reference = mappings_data[mappings_data['Cluster ID'].isin(ref_clusters)]
ref_id = set(reference['Cat ID'])
predictions = pd.DataFrame([(name, sim, 'Seed' if catid == idx_to_catid(randseed) else catid in ref_id) for catid, name, sim in knn_results],columns=['Category','Similarity','In reference'])
display(predictions)
display(reference)